### 抓取微博发言

- 作者：DGC'Idea
- 版本：V0.3   
- 更新日期：2018年4月9日

----------

- 研究目的：获取关注微博博主的发言；
- 研究方法：从微博手机主页爬取所有发言。

In [1]:
#-*- coding: utf-8 -*-

# 基本
import numpy as np
import pandas as pd

# 系统
import os

# 网络
import urllib.request
import requests
import json
import re

# 日期、时间
import time
from datetime import timedelta,date,datetime

# 网页解析
from bs4 import BeautifulSoup

# 自定义
import sys
sys.path.append('/home/jquser')

from dm import *
from tl import exists_file


# 数据目录
_data_path='../../Data/'
# 数据引擎
_data=Csv(_data_path)


#设置代理IP
proxy_addr="122.241.72.191:808"

#性别
Gender={'m':'男','w':'女'}

#性别
Accreditation={'True':'是','False':'否'}

#使用代理访问页面
def use_proxy(url,proxy_addr):
    req=urllib.request.Request(url)
    req.add_header("User-Agent","Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36\
                   (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0")
    proxy=urllib.request.ProxyHandler({'http':proxy_addr})
    opener=urllib.request.build_opener(proxy,urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    data=urllib.request.urlopen(req).read().decode('utf-8','ignore')
    return data



#获取微博账号的用户基本信息，如：微博昵称、微博地址、微博头像、关注人数、粉丝数、性别、等级等
def get_weibo_userInfo(id):
    #url地址
    url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id
    #使用代理访问页面
    data=use_proxy(url,proxy_addr)
    #解析数据
    content=json.loads(data).get('data')
    #头像地址
    image_url=content.get('userInfo').get('profile_image_url')
    #说明
    description=content.get('userInfo').get('description')
    #主页地址
    profile_url=content.get('userInfo').get('profile_url')
    #是否认证
    verified=content.get('userInfo').get('verified')
    #关注数
    follow_count=content.get('userInfo').get('follow_count')
    #昵称
    name=content.get('userInfo').get('screen_name')
    #粉丝数
    followers_count=content.get('userInfo').get('followers_count')
    #性别
    gender=content.get('userInfo').get('gender')
    #等级
    urank=content.get('userInfo').get('urank')
    statuses_count=content.get('userInfo').get('statuses_count')
    #显示信息
    return(
          name,
          Gender[gender],
          profile_url,
          Accreditation[str(verified)],
          urank, 
          follow_count,
          followers_count,
          statuses_count,
          description,    
         )


#获取微博主页的containerid，爬取微博内容时需要此id
def get_containerid(url):
#     data=use_proxy(url,proxy_addr)
    #直接请求模式
    data=requests.get(url).text 
    content=json.loads(data).get('data')
    for data in content.get('tabsInfo').get('tabs'):
        if(data.get('tab_type')=='weibo'):
            containerid=data.get('containerid')
    return containerid


#获取微博的更进一步信息：确切时间、完整正文
def get_weibo_text(url):
    #代理模式
#     data=use_proxy(url,proxy_addr)
    #直接请求模式
    data=requests.get(url).text 
    #剥离不相关信息
    data=data.split('var $render_data = [')[1].split('][0] ')[0]
    #取得json信息
    content=json.loads(data).get('status')
    #微博正文
    text=content['text']
    #完整时间
    created_at=content['created_at']
    #转换时间格式（原格式比较特别）
    created_at=str(datetime.strptime(created_at.replace('+0800',''),'%a %b %d %H:%M:%S %Y'))    
    #返回正文和时间
    return text,created_at
    
    
#获取微博内容：正文、详情页地址、点赞数、评论数、转发数等
def get_weibo_data(user_id,start_date=None,First=False):
    #页数计数器
    page=0
    #微博计数器
    count=0
    #数据表
    data_list=[]
    #微博名
    name=id_all_list[user_id]
    #退出标记
    loop=True
    #url（微博用户信息网址）
    url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+user_id
    #获得完整id
    containerid=get_containerid(url)
    #页面循环
    while loop:
        #开启错误抓取
        try:
            # 页面数累计            
            page+=1    
            weibo_url='https://m.weibo.cn/api/container/getIndex?type=uid&value='\
                       +user_id+'&containerid='+containerid+'&page='+str(page)
  
            # 代理模式
#             data=use_proxy(weibo_url,proxy_addr) 
            #直接请求模式
            data=requests.get(weibo_url).text 
            # 取得josn数据
            content=json.loads(data).get('data')
            # 取得微博信息
            cards=content.get('cards')
            
            # 判断是否为空数据
            if(len(cards)==0):
                #退出循环
                break   
                
            #遍历当前页面所有微博  
            for card in cards:
                card_type=card.get('card_type')
                
                # 非微博发言
                if(card_type!=9):
                    continue
                    
                # 微博内容    
                mblog=card.get('mblog')
                
                #去除置顶微博
                if len(mblog.get('edit_at',''))!=0 :
                    continue
                    
                #微博实际网址    
                scheme=card.get('scheme')  
                #微博实际正文、发布时间    
                text,created_at=get_weibo_text(scheme)
                
                if created_at<=start_date:
                    loop=False
                    break
                    
                #微博数累计
                count+=1
                print('正在获取：%s %s 的微博，已获取 %s 条'%(name,created_at,count),end='\r')
                #添加到数据列表
                data_list.append([created_at,text,
                                mblog.get('attitudes_count'),mblog.get('comments_count'),mblog.get('reposts_count'),                                
                                scheme,False,
                                ])
            #延时
            time.sleep(10.0)
        #错误处理
        except Exception as e:
            print('错误：%s'%(e),end='\r')
            if '418' in str(e):
                #遇到418错误
                time.sleep(3.0)
                pass
            else:   
                #其它错误，延时后继续
                time.sleep(1.0)
                pass   
            
    if len(data_list)!=0:
        #数据表整理                
        columns_list=['created_at','text','attitudes_count','comments_count','reposts_count','scheme','sended']
        df=pd.DataFrame(data=data_list,columns=columns_list) 
        #返回数据
        return df
    else:
        return None
    
#增量更新
def update_weibo_data(id_list): 
    #遍历所有id
    for user_id in id_list:
        #微博名
        name=id_all_list[user_id] 
        df=pd.DataFrame()
        #判断文件是否存存在
        if exists_weibo_file(user_id):    #+'test'        
            #增量更新         
            #从cvs文件中导入数据
            df=read_weibo_data(user_id)
            #获得增量开始日期
            update_date=df['created_at'].iloc[0]  
        else:
            #初次计算
            update_date=None
            
        # 获取数据    
        update_df=get_weibo_data(user_id,update_date) 
        
        # 判断是否获取到数据
        if not update_df is None:
            print('共计更新：%s %s 条微博'%(name,len(update_df)))
            #拼接新旧数据，并重置索引
            df=pd.concat([update_df,df],ignore_index=True) 
            #return update_df
            #保存数据到cvs 
            save_weibo_data(user_id,df)
        
#检测文件      
def exists_weibo_file(user_id,data_root=_data_path):
    #cvs文件名
    data_file='%s%s.csv'%(data_root,'msg_weibo_'+user_id)
    return exists_file(data_file)


#读取数据        
def read_weibo_data(user_id):
    #文件名
    data_file='msg_weibo_'+user_id
    #从cvs文件恢复数据
    df=_data.read(data_file)
    return df

#保存数据        
def save_weibo_data(user_id,df):
    #文件名
    data_file='msg_weibo_'+user_id
    #保存到cvs文件
    _data.save(data_file,df,append=False)
    

#id列表
id_all_list={
    '5687069307':'ETF拯救世界',
#     '1669616825':'程老湿爱吐槽',
#     '1254381361':'闲来一坐'
        }

# print(' 已准备好...')   

数据引擎：运行于研究


## 1.更新数据

In [2]:
#更新数据
df=update_weibo_data(id_all_list)
df

共计更新：ETF拯救世界 263 条微博 range:00:04 的微博，已获取 263 条


## 2.设置当前ID

In [3]:
#获取当前微博ID信息
# get_weibo_userInfo(id_all_list.keys().tolist()[1])
user_id='5687069307'
(user_name,
gender,
profile_url,
verified,
urank, 
follow_count,
followers_count,
statuses_count,
description)=get_weibo_userInfo(user_id)
print(
      " 微博昵称："+user_name+"\n"+
      " 性　　别："+gender+"\n"+
      " 主页地址："+profile_url+"\n"+
      " 是否认证："+verified+"\n"+
      " 等　　级："+str(urank)+"\n"+  
      " 关注人数："+str(follow_count)+"\n"+
      " 粉 丝 数："+str(followers_count)+"\n"+
      " 微 博 数："+str(statuses_count)+"\n"+
      " 说　　明："+description+"\n"       
     )

 微博昵称：ETF拯救世界
 性　　别：男
 主页地址：https://m.weibo.cn/u/5687069307?uid=5687069307&luicode=10000011&lfid=1005055687069307
 是否认证：是
 等　　级：39
 关注人数：304
 粉 丝 数：216200
 微 博 数：6982
 说　　明：微信公众号：chinaetfs 雪球ID：ETF拯救世界



## 3.查看最近发言
- 查看今天、昨天、本月发言；
- 去除了发言中的连接等无关信息；

### 3.1.今天的发言

In [10]:
from bs4 import BeautifulSoup


#查看最新发言      
def show_weibo_text(user_id,day='今天',url=False,data_root='./'):       
    #从cvs文件恢复数据
    df=read_weibo_data(user_id)
    df['time']=df['created_at']
    df['created_at']=df['created_at'].astype('str').str[0:10]
    if day=='今天' :
        day=pd.datetime.today().strftime('%Y-%m-%d')
        df=df[df['created_at']==day]
    elif day=='昨天':
        day='昨天'
        day=(pd.datetime.today()-timedelta(1)).strftime('%Y-%m-%d')        
        df=df[df['created_at']==day]
    elif day=='本月' :
        day=pd.datetime.today().strftime('%Y-%m')+'-01'   
        df=df[df['created_at']>=day]
    elif day=='上半年' :
        day_start=pd.datetime.today().strftime('%Y')+'-01-01'   
        day_end=pd.datetime.today().strftime('%Y')+'-06-30' 
        df=df[(df['created_at']>=day_start) & (df['created_at']<=day_end)]
    elif day=='下半年' :
        day_start=pd.datetime.today().strftime('%Y')+'-07-01'   
        day_end=pd.datetime.today().strftime('%Y')+'-12-31' 
        df=df[(df['created_at']>=day_start) & (df['created_at']<=day_end)]    
    else:
        return
    df=df.sort_values(['created_at'],ascending=True)
    if len(df)==0:
        print('无内容...')
        return
    for i in range(len(df)): 
        text=df['text'].iloc[i]
        print(df['created_at'].iloc[i]) 
        soup=BeautifulSoup(text,'html.parser')
        print (''.join([item for item in soup.stripped_strings]))
        if url:
            print(str(df['scheme'].iloc[i]))
        print("\r")  

show_weibo_text(user_id,"今天")

2019-03-19
均衡配置有一点不好，就是今天这种情况你不知道组合到底是涨是跌……

2019-03-19
40%了。所有关注的指数排名第三。位列券商和信息后面。为什么有人的视野那么小。//@ETF拯救世界:回复@飘落依晨:传媒今年涨了37%。你说这话亏心不//@飘落依晨:中证传媒的表现真的失望 啊

2019-03-19
$养老产业 sz399812$   反弹新高。

2019-03-19
下一次礼物压力位是 +50%。送的东西应该是绝大多数人喜欢的。无论男女。价值也会比ps4高很多很多。大家一起发功意念加成早日达成吧。

2019-03-19
回复@青山亦旧:你这又是贴标签了。私募里面也有牛人，而且比例肯定比散户多。只是不是所有私募都是牛人。//@青山亦旧:对了，私募就是干高买低卖的，谢谢了啊//@ETF拯救世界:回复@i云腾致雨:你是不是对私募有什么误会？2015年以来几千个私募产品清盘。千万不要因为一个人的身份就认为他应该是什么样的

2019-03-19
回复@i云腾致雨:你是不是对私募有什么误会？2015年以来几千个私募产品清盘。千万不要因为一个人的身份就认为他应该是什么样的//@i云腾致雨:跟踪一个大V，2000多点减仓到三成，但这两天要加到七成仓位，私募也高买低卖啊？

2019-03-19
回复@你好大麦茶:他在想要不要先跳车躲回调//@你好大麦茶:你都在车上了，还担心啥？就算再跌下去都可以继续买啊，计划里还有现金啊//@ETF拯救世界:回复@欲雨鱼鱼愉:行情总在犹豫中开始。如果所有人都认可牛市，那就是要见顶了

2019-03-19
回复@欲雨鱼鱼愉:行情总在犹豫中开始。如果所有人都认可牛市，那就是要见顶了//@欲雨鱼鱼愉:e大怎么看还有一批“技术派”认为没有震荡筑底的过程现在不是牛市的//@ETF拯救世界:回复@给你买糖吗:没有为这个郁闷过。一般不借，借了就做好要不回来的准备了

2019-03-19
回复@给你买糖吗:没有为这个郁闷过。一般不借，借了就做好要不回来的准备了//@给你买糖吗:想问老大一个问题诶，老大试过借钱给别人要不回来的时候么就心里很憋屈…会怎么调节呢？按照：借给他的钱比不上一天的涨跌这样调节么

2019-03-19
回复@蜀中弟子:朋友去年底送钱的多了，没几个人相信并且真的买。这也是他自己的运势到了//@蜀中弟子:回

### 3.2.昨天的发言

In [14]:
show_weibo_text(user_id,"昨天")

2018-11-02
回复@云心皓月:我不关心别人怎么看。我有自己判断顶底的标准。//@云心皓月:E大，丹神和飘仙确认2449是十年一遇的大底了。

2018-11-02
哥们有点意思。为什么告诉电视台让全国人民知道初中女同学过来同住呢……

2018-11-02
涨了，赚钱了，不用激动，也没理由激动。为什么，因为你买入的时候就应该知道这是97.1%会发生的。这么大概率的事情发生了，你激动什么？每天你看见太阳升起来会激动吗。跌了，浮亏了，同样不用担心。为什么。因为你知道自己买的是便宜的，不会死的好东西。跌就跌呗。97.1%概率以后肯定挣钱啊。还不趁着越来越便宜买买买。综上。佛系买指数，波澜不惊，笑看风云。

2018-11-02
三张图告诉你为什么我看到头条的评论一点都不生气，还感觉很欢乐。

2018-11-02
回复@黄连树:不。你看头条的人虽然喷，但是并不会很恶毒，甚至有点可爱。球球上有些人写出的文字，其恶毒程度，已经不仅是侮辱人格。甚至有人会组队来喷。那是一个江湖，早早归隐才是王道。

2018-11-02
反转还是反弹，不是猜出来的，是走出来的。观察就行，不要预测。根据我的体系，确认反转还需要有两个条件满足。这两个条件满足了，反转的或然率超过80%。走着看呗。踏踏实实的。

2018-11-02
回复@OO大月半:这是没办法的。这就是所谓右侧的弊端。趋势明确反转必然已经是有一段涨幅了。左侧在下跌中买入，会套。右侧在上涨中买入，会比最低点高。没有任何方法能保证能买在最低点。没有任何方法，没有。//@OO大月半:满足反转条件的时候，估计已经从底部上来很多了吧

2018-11-02
两个人都有问题。情绪完全控制不住。以后打司机的行为能不能以危害公共安全罪判五年罚50万？并且大力宣传，看谁还敢。司机方面，我就不说了，不知道最后的方向盘是怎么回事……最无辜的，是这一车乘客。

2018-11-02
回复@站在空气稀薄地带:哪有的事儿呢//@站在空气稀薄地带:说，你是不是提前知道主席召开的会议到行事历了

2018-11-02
//@王志安: 公布的视频太短了，既然已经恢复了sd卡，为什么不能多公布一些内容呢？最关键的是司机撞桥栏和之后的身体姿势和神态，这能帮助分析出，变道撞桥坠江到底是司机的失误还是因为发怒之后的有意为之。目前这个视频并不支持调查结论。我认为这个公

### 3.3.本月的发言

In [11]:
show_weibo_text(user_id,"本月")

2019-03-07
开始看留言，1400条。

2019-03-07
收盘跑赢2.3%

2019-03-08
我再说一次，这是骗子。说过很多次了。我只有一个公众号 ：chinaetfs  长赢指数投资

2019-03-08
第二题，卖出的这份什么价格买回来？

2019-03-08
$中证500 sz399905$不会第一次出手卖就卖在阶段最高点了吧。瞎蒙也可以？

2019-03-08
小贴士：股票价格从1 2 3 4……涨到10，你一路不敢买。回调到9你认为很便宜了下手买入。这样很危险。

2019-03-08
回复@fly小小牛:其中之一是观察全指信息。只有在真正的牛市中，科技板块才会领涨。//@fly小小牛:有啥信号📶，群情汹涌？

2019-03-08
回复@以后做快乐健康的小女人:2015年5月，手上拿到1000万拆迁款。他不去买股票基金应该去干嘛呢？——有钱不是投资的理由，好机会才是！！！

2019-03-08
今日锦鲤@小静姐carol选的是这个。能让别人开心自己也会很开心。

2019-03-08
我个人希望这一份能和趋势仓位一起买回来。希望都是好的，未必能实现啊。给点力，大盘。

2019-03-08
上涨趋势中，追高的人遇到调整很容易割肉。割肉以后，又涨起来。追还是不追，就很尴尬。追了又跌，不追就一路涨。当然，也有时候他们不割。那也行，不割就一路跌下去套死了。

2019-03-08
今天300又被500干了1.3%。全年已经落后500达到6%以上。//@ETF拯救世界:收盘跑赢2.3%

2019-03-08
卖了一份而已，至于这么大反应吗…

2019-03-08
昨天收盘还差0.4，我的游戏机保住了…不过，下次会更快到20%。

2019-03-08
今天的节日礼物会送给这里面排名最靠前的女性朋友。我想这个榜单里有坏人的可能性不大。至于送什么，我会和她商量。总之希望她满意。也希望这里所有的女性朋友节日快乐！

2019-03-09
有没有人看过《双瞳》？跟这个几乎一样。我强推的一部电影，个人非常喜欢！//@蓝鲸财经记者工作平台:硬核//@霜叶:人生后花园，地下CBD……

2019-03-09
回复@低级工程师Rick:我不敢。会有人说我借推荐这个赚钱。什么都不做还祸从天上来，这种事我还是别干了。//@低级工程师Rick:看来E大

### 3.4.上半年的发言

In [12]:
show_weibo_text(user_id,"上半年")

2019-01-02
遗产税我个人支持。问题在于怎么才能让这个从富人那里拿到的税真正用在穷人身上。

2019-01-02
没为母校做什么，很惭愧

2019-01-02
2001-2005，上证指数从2245跌到998，跌幅56%.2007-2012，不用上证了，因为被大股票绑架。用深综指，从1500跌到770，跌幅50%。（08年底跌幅70%）这一轮深综指已经杀了60%。如果到05年底估值，等于是从最高点杀67%。你可以有一万种理由说这次不一样，你看我搭理不搭理你。

2019-01-02
回复@追求确定性2:快吗？从2001年跌到2005整整4年。从2007到2012年底又是4年半。一轮牛市顶部到熊市大底都是4年起。你回头看觉得很快，当你置身其中，比如现在这轮才三年多，就感受完全不同了。//@追求确定性2:哦，我竟然不知道05年低于08年，那时一轮好快，节约时间。

2019-01-02
今天不出意外会跌破10月底的全市场估值。距离2008年最低点还有11%，距离2005年最低点还有17%。2008年之后是一年翻番。2005年之后是两年六倍。2019年之后会怎样，咱们拭目以待。

2019-01-02
回复@目标300m:张嘴就来。医药从15500跌下来，我们的成本在这里，从未追高，太高之说从何而来？服了，扯淡之前先看看数据。拉黑为敬。评论配图//@目标300m:6000以下再买吧。你之前买得太高了。

2019-01-02
回复@业余金融学者:没事。就让贫富差距继续扩大吧。穷人越来越穷，毫无上升的机会，最终来一次gm，均贫卡一出谁与争锋，那样就都开心了。

2019-01-02
回复@佳期如梦梦梦:这么明显的下跌趋势，当然是给空间了。2018年我们分别在2、8、10月买了三份。不能把建仓筹码聚集在一个区域，否则毫无意义。//@佳期如梦梦梦:很想知道E大啥时候买医药//@ETF拯救世界:下次买入应该在7900左右。

2019-01-02
珍惜眼前人。

2019-01-02
这股票前期低点撑不住的话就麻烦。权重又大，大指数又是一波。

2019-01-02
你可能不知道，去年我们第一重仓基金中证红利是所有我监测的十几只指数中涨幅第一的。年涨幅是-16%……

2019-01-02
我立个flag。2019年A股极大极大概率收红。不投机取巧，我说的是绝大多数指

### 3.5.下半年的发言

In [13]:
show_weibo_text(user_id,"下半年")

无内容...


## 4.查看原始数据
- 查看DataFrame原始数据。

In [4]:
#查看原始数据    
def show_weibo_data(user_id,data_root='./'):
    #从cvs文件恢复数据
    df=read_weibo_data(user_id)
    df=df.rename(columns={'created_at':'发布时间','text':'内容',
                            'attitudes_count':'点赞','comments_count':'评论','reposts_count':'转发',
                            'scheme':'地址'})
    return df

df=show_weibo_data(user_id)
df

,发布时间,内容,点赞,评论,转发,地址,sended
0,2019-07-24 14:36:35,钱不禁花。<br /><br />一个感觉，不一定对。,2448.0,1021.0,38.0,https://m.weibo.cn/status/HEWIli7oM?mblogid=HE...,False
1,2019-07-23 21:56:03,去年跑赢20%不说了？谁能每个时间段都跑赢指数，上帝？,643.0,182.0,27.0,https://m.weibo.cn/status/HEQadtYfY?mblogid=HE...,False
2,2019-07-23 20:53:07,还想去US的朋友慎重。美签的时候签证官会非常关注去过Iran这件事。,583.0,164.0,13.0,https://m.weibo.cn/status/HEPKFDJLk?mblogid=HE...,False
3,2019-07-23 20:42:58,白银继续反弹新高。,478.0,87.0,6.0,https://m.weibo.cn/status/HEPGz0fEn?mblogid=HE...,False
4,2019-07-23 18:08:33,新裤子的彭磊当年不就是为了买房省中介费要去链家当中介吗……,683.0,198.0,51.0,https://m.weibo.cn/status/HEOFSBYXZ?mblogid=HE...,False
5,2019-07-23 18:00:48,震惊！<br /><br />过去13天我只发了10条微博！<br /><br />这种亲者...,1697.0,506.0,8.0,https://m.weibo.cn/status/HEOCJkE9N?mblogid=HE...,False
6,2019-07-23 14:45:40,多看少动。<br /><br />不看也没差。<br /><br />找点别的事分散一下精力。,1424.0,460.0,18.0,https://m.weibo.cn/status/HENlwwAVK?mblogid=HE...,False
7,2019-07-22 13:40:45,白银近期涨幅大于黄金，但年度涨幅看依然落后很多。A股市场上没有合适的白银基金，有的一只是做期...,1003.0,413.0,68.0,https://m.weibo.cn/status/HEDuGd9ft?mblogid=HE...,False
8,2019-07-22 10:51:05,已公布的14家公司半年报，二季度平均利润增长超过30%。<br /><br />但是家数太少...,1353.0,690.0,44.0,https://m.weibo.cn/status/HECnODwSt?mblogid=HE...,False
9,2019-07-21 17:40:48,当代中国的另一面。有点好笑，有点感动，也有点难过。,851.0,395.0,100.0,https://m.weibo.cn/status/HEvDD0BAB?mblogid=HE...,False


## 5.整理数据
- 获取微博完整信息：完整日期、完整内容。

In [10]:
#整理数据 
def dig_weibo_data(user_id,data_root='./'):
    df=read_weibo_data(user_id)
    #计数器
    count=0   
    #遍历所有数据
    for i in range(len(df)): 
        #开启错误抓取
        try:
            if len(df['created_at'].iloc[i])<=10:
                #计数器累加
                count+=1
                print(' 正在整理：%s 的微博...'%(df['created_at'].iloc[i]),end='\r')
                text,day=get_weibo_text(df['scheme'].iloc[i])
                df['created_at'].iloc[i]=day
                df['text'].iloc[i]=text
        #错误处理
        except Exception as e:
            print(' 错误：%s'%(df['scheme'].iloc[i]),end='\r')
            pass
    #保存数据到cvs 
    save_weibo_data(user_id,df)
    return df

df=dig_weibo_data(id)
df=df[df['created_at'].astype('str').str.len()<=10]   
df
print(len(df))

TypeError: Can't convert 'builtin_function_or_method' object to str implicitly

## 6.生成文本文件
- 生成文本文件，可作为下载后阅读使用；
- 已去除了内容中的多余链接等无关信息。

In [ ]:
#生成txt文件      
def write_weibo_text(user_id,data_root='./'):
    #从cvs文件恢复数据
    df=read_weibo_data(user_id)
#     if exists_weibo_file(user_id):  
#         os.remove(file_path)
    for i in range(len(df)): 
        print('-正在整理，第%s条微博...'%(i),end='\r')
        
        text=df['text'].iloc[i]
        soup=BeautifulSoup(text,'html.parser')
        text=''.join([item for item in soup.stripped_strings])
               
        with open(file_path,'a',encoding='utf-8') as fh:                      
            fh.write(
                 "发布时间："+str(df['created_at'].iloc[i])+"\n"
                 +"微博内容："+text+"\n"
                 +"微博地址："+str(df['scheme'].iloc[i])+"\n"
                 +"\n"   
                )

write_weibo_text(user_id,)

## 7.发送邮件
- 发送某个月的所有发言到邮箱；

In [5]:
import sys

#引入自定义库
sys.path.append("/home/jquser/")
from tl_mail import *

#发送某月数据
def send_weibo_data(user_id,user_name):
    #检查是否已存在数据
    if not exists_weibo_file(user_id):
        print('未找到 %s 的微博发言数据表！'%user_id)
        return
    #读取数据
    df=read_weibo_data(user_id)
    #计算起止年
    start_year=int(df.created_at.iloc[-1][:4])
    end_year=int(df.created_at.iloc[0][:4])+1
    #月
    m_list=['01','02','03','04','05','06','07','08','09','10','11','12'] 
    #当前月份
    current_month=pd.datetime.today().strftime('%Y-%m')
    #已发送标记
    issend=False
    #遍历年、月
    for y in range(start_year,end_year):
        for m in m_list:
            month=str(y)+'-'+m
            #判断是否为当月，当月不发送
            if current_month==month:
                continue
            #按照年月筛选    
            temp_df=df[(df.created_at.str[0:7]==month) & (df.sended==False)]
            temp_df=temp_df.sort_values('created_at')
            if len(temp_df)==0:
                continue  
            #生成月邮件正文    
            message=''
            for i in range(len(temp_df)):
                #日期
                message+='<p>%s</p>'%temp_df.created_at.iloc[i]  
                #url
                message+='<p>%s</p>'%temp_df.scheme.iloc[i] 
                #正文    
                message+='%s</br></br><hr /></br>'%temp_df.text.iloc[i] 
            #邮件主题    
            subjiect='%s年%s月（微博发言 %s）'%(y,m,user_name)
            #内容标题
            title='<h2>%s</h2></br><hr />'%(subjiect)
            #邮件内容
            message=title+message
            #去除多余的行
            #message=message.replace('<br/><br/>','').replace('</p></br>','</p>').replace('<p><br/></p>','')
            try:
                #发送邮件
                send_html_qqmail(subjiect,message)
                #标记已发送
                if len(temp_df)==1:
                    df.sended.iloc[temp_df.index[0]]=True
                else:
                    df.sended.iloc[temp_df.index[-1]:temp_df.index[0]+1]=True
                issend=True    
                print(' %s 的发言发送成功！'%(subjiect),end='\r')
                #延时5秒，防止被踢
                time.sleep(3)
            except:
                print(' %s 的发言发送失败！'%(subjiect),end='\r')
    #如果有发送，则保存            
    if issend:
        save_weibo_data(user_id,df)

        
send_weibo_data(user_id,user_name)  

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [28]:
pd.set_option('display.max_rows',2000)
df=read_weibo_data(user_id)
# del df[df.index==702] 
#del df[(df['created_at']==False)] 
#df.drop(axis=0,labels=['False'],inplace=True) 
#df[(df['created_at']>='2018-09-01') & (df['created_at']<='2018-09-30')] 
#df[(df['created_at']>='2018-09-01') & (df['created_at']<='2018-09-30')].sended=False
#save_weibo_data(user_id,df)
df

,created_at,text,attitudes_count,comments_count,reposts_count,scheme,sended
0,2019-03-19 15:48:12,均衡配置有一点不好，就是今天这种情况你不知道组合到底是涨是跌……,525.0,289.0,11.0,https://m.weibo.cn/status/HlDUliiIt?mblogid=Hl...,False
1,2019-03-19 10:10:33,40%了。所有关注的指数排名第三。位列券商和信息后面。为什么有人的视野那么小。//<a hr...,501.0,125.0,11.0,https://m.weibo.cn/status/HlBHio4LF?mblogid=Hl...,False
2,2019-03-19 09:53:31,$养老产业 sz399812$ 反弹新高。,447.0,52.0,1.0,https://m.weibo.cn/status/HlBAnk2su?mblogid=Hl...,False
3,2019-03-19 09:49:02,下一次礼物压力位是 +50%。<br /><br />送的东西应该是绝大多数人喜欢的。无论男...,1469.0,662.0,26.0,https://m.weibo.cn/status/HlByyqyN2?mblogid=Hl...,False
4,2019-03-19 09:28:25,回复<a href='/n/青山亦旧'>@青山亦旧</a>:你这又是贴标签了。私募里面也有牛...,365.0,38.0,2.0,https://m.weibo.cn/status/HlBqc8WQB?mblogid=Hl...,False
5,2019-03-19 09:24:04,回复<a href='/n/i云腾致雨'>@i云腾致雨</a>:你是不是对私募有什么误会？2...,308.0,32.0,7.0,https://m.weibo.cn/status/HlBoqezGK?mblogid=Hl...,False
6,2019-03-19 09:21:56,回复<a href='/n/你好大麦茶'>@你好大麦茶</a>:他在想要不要先跳车躲回调//...,299.0,41.0,1.0,https://m.weibo.cn/status/HlBnz2NtB?mblogid=Hl...,False
7,2019-03-19 09:18:01,回复<a href='/n/欲雨鱼鱼愉'>@欲雨鱼鱼愉</a>:行情总在犹豫中开始。如果所有...,329.0,40.0,28.0,https://m.weibo.cn/status/HlBlYmsDS?mblogid=Hl...,False
8,2019-03-19 07:49:13,回复<a href='/n/给你买糖吗'>@给你买糖吗</a>:没有为这个郁闷过。一般不借，...,352.0,101.0,32.0,https://m.weibo.cn/status/HlALVgOyr?mblogid=Hl...,False
9,2019-03-19 07:20:37,回复<a href='/n/蜀中弟子'>@蜀中弟子</a>:朋友去年底送钱的多了，没几个人相...,322.0,76.0,4.0,https://m.weibo.cn/status/HlAAjpMew?mblogid=Hl...,False
